<a href="https://colab.research.google.com/github/chonholee/tutorial/blob/main/bigdata/BigDataII_14_QA_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **はじめに**

---
D-Wave Systemsが提供しているOcean SDKをインストール

※「Quantam Aneeling for You」を参照
https://altema.is.tohoku.ac.jp/QA4U/

In [ ]:
pip install dwave-ocean-sdk

上記のコードを実行すると、**RESTART RUNTIME** というボタンが表示されると思います。

そのボタンを押して [ランタイムの再行動] をしておきましょう。

これでD-Wave Systemsの量子アニーリングマシンを利用することができる準備が整いました。

## **量子アニーリングマシンへの入力**

量子アニーリングマシンでは一体どのようなことができるのか。

できることはある意味数が少なく「組合せ最適化問題を解く」ということが主な用途となります。

しかもその組合せ最適化問題のうち、
シンプルな形を持つ次のような関数を最小化するだけという不器用さです。
\begin{equation}
E({\bf x})  = \sum_{i=1}^{N} \sum_{j=1}^N Q_{ij} x_i x_j
\end{equation}

このような関数を最小化する問題のことを**制約なし2値の2次計画問題**と言います。

英語でQuadratic unconstrained binary optimization（QUBO）と呼びます。

最小化される対象となる関数のことを**コスト関数**と呼びます。

* ここで$x_i$は$0$か$1$を取る２値の変数です。変数というのは、状況によって値を変えるものですので、ここではどんな値を取るのか、色々な可能性があります。

* $Q_{ij}$はQUBO行列と呼び、その値によって様々な組合せ最適化問題を表すことができます。

### **高校数学との接点**
シグマ記号（$\Sigma$）が登場して驚く読者もいると思います。
ただシグマ記号を始め、数学の記号はじっくり眺めるとちゃんと読むことのできるものであり、
その読み方さえ習えば怖いものではないのです。
それこそ辞書を片手に英語を読むようなものです。だからさっと見せられて翻訳の済む前に次の話に行くと、
わからないままに進むので嫌な気分だけが記憶に残り、苦手意識に変わります。

そこでじっくりとシグマ記号に慣れ親しむことにしましょう。
シグマ記号は
**「下についた文字の始まりから上に書いてある終わりまで、その数値を変えながら、どんどん足していくぞ」**
という宣言をするものです。だから**たくさん足し算するぞ**と言っているだけです。


例えば先ほどのコスト関数は、、、
\begin{equation}
\sum_{i=1}^N \sum_{j=1}^N Q_{ij}x_ix_j = Q_{11}x_1x_1 + Q_{12}x_1x_2 + \cdots + Q_{1N}x_1x_N + Q_{21}x_2x_1 + \cdots + Q_{NN}x_Nx_N
\end{equation}
というぐあいに、$i$に$1$を入れてから$j$に$1$を入れて$2$を入れて$3$を入れて、それらを全て足す、ということをしますよという意味です。

### **早速量子アニーリングマシンを使ってみよう！**

いきなり数学の話が長々と続いてしまっては面白くなくなりますよね。
早く量子アニーリングを実行したいと思っているはずですから。まずは動かすということをやってみましょう！
まずは皆さん、D-Wave Systems社のWebページからアカウントを作成しましょう。
実は1分間だけは誰でも無料で使えます。

### **アカウント登録**
https://cloud.dwavesys.com/leap/
にアクセスしてください。

メールアドレスや氏名・所属情報を記入してご自身のアカウントを作成してください。

そのアカウントでログインしたのちに、画面左下にあるAPI tokenをコピーしてください。

### **簡単な問題を投入してみよう！**

それではアカウント登録が済んだら、そこで得られたご自身のAPI keyを利用して、量子アニーリングマシンに問題を投じてみましょう。先ほど紹介したように量子アニーリングマシンに問題を投じるというのは、QUBO行列をカナダにある量子アニーリングマシンに送信するという意味です。
だからQUBO行列さえ用意することができれば良いわけです。
やってみましょう。

In [ ]:
import numpy as np

N = 10
QUBO = np.random.randn(N**2).reshape(N,N)

まずは簡単な例として、10×10行列によるQUBOを考えてみます。
np.random.randn()は、平均0、分散1のガウス分布に従う乱数を生成するという関数です。
それをN**2=100個作ったのち、reshape(N,N)として、10×10の行列の形にします。

これでQUBO行列が用意できました。
このQUBO行列には正負の値がでたらめに入っています。
それぞれの値を見ることもできます。

In [ ]:
print(QUBO[0][1])

$Q_{ij}$の値が正であれば、コスト関数の一部分を注目すると$Q_{ij}x_ix_j$を小さい値にするには、$x_ix_j = 0$になるのが良い。

- $x_i=0$, $x_j=1$
- $x_i=1$, $x_j=0$
- $x_i=0$, $x_j=0$

という3通りが選ばれることになる。逆に$Q_{ij}$が負の値をとると$x_ix_j=1$となるのが良い。
- $x_i=1$,$x_j=1$

単純に$ij$のペア1つだけに注目するのであれば、こうした傾向を考えていけば良い。
しかし他にも$i$にかかる別のペアから他の向きになることを要求される可能性もある。
さらに$Q_{ii}x_ix_i$というQUBO行列の対角成分$Q_{ii}$の値次第で、$x_i$がまた影響を受ける。$Q_{ii}$が正の値を取ると$x_i=0$になった方が良いし、$Q_{ii}$が負の値を取ると$x_i=1$になった方が良い。
以上のような様々な影響がかかり、どの向きを向いたら良いのかがすぐにはわからない悩ましい問題となる。組合せ最適化問題にはそうした悩ましい状況が往々にして生じる。
こうした問題に対して、D-Wave Systemsの量子アニーリングマシンで答えを出してみよう。

まず先ほど登録したアカウントからAPI tokenを入手します。
ログインした後の画面において左側にあるボタンからAPI tokenをコピーしましょう。
そのtokenを以下の**の代わりに入力してください。

In [ ]:
token = '**'  # <--- 個人のAPI tokenを使用
endpoint = 'https://cloud.dwavesys.com/sapi/'

これでD-Wave Systems社の量子アニーリングマシンを利用する環境が整いました。
続けて量子アニーリングマシンを呼び出す関数をいくつか用意します。

In [ ]:
from dwave.system import DWaveSampler, EmbeddingComposite

dw_sampler = DWaveSampler(solver='Advantage_system4.1', token=token, endpoint=endpoint)

まず from dwave.systemとあるのはOcean SDKのdwave.systemの中からということです。

**import DWaveSampler**でDWaveSamplerという関数を呼び出しており、

**import EmbeddingComposite**でEmbeddingCompositeという関数を呼び出しています。

この呼び出した関数のうちDWaveSamplerは、量子アニーリングマシンにアクセスするための関数です。
solver=というオプションで、どの量子アニーリングマシンを使うのかを指定することができます。

最新式の量子アニーリングマシンはD-Wave Advantageであり、それを利用する場合には

**solver = 'Advantage_system4.1'**としましょう。

前の形式の量子アニーリングマシンであるD-Wave 2000Qを利用する場合には

**solver = 'DW_2000Q_6'**としましょう。

それぞれ利用したいものを指定すれば準備完了です。

量子アニーリングマシンの中にはQPU（Quantum Processing Unit）と呼ばれる部品があり、
そこに搭載された超伝導量子ビットによる回路のパラメータを調整して$Q_{ij}$を与えます。
回路の設計上、$Q_{ij}$で指定された構造を作ることができない場合には、他の回路や量子ビットをうまく組み合わせるエンベッドという作業をします。
その作業をするには先ほど読み込んだ関数のうちEmbeddingComposite()という関数を利用することができます。

In [ ]:
sampler = EmbeddingComposite(dw_sampler)

エンべディングを終えると、超伝導量子ビットにどの$x_i$を割り当てるか、
どのように$Q_{ij}$を割り当てるのかが決められた新しいsamplerに置き換わります。
これを利用することで,先ほど用意したQUBO行列に基づく組合せ最適化問題を量子アニーリングマシンに解かせることができます。

In [ ]:
sampleset = sampler.sample_qubo(QUBO, num_reads=10)

sampler.sample_quboというのが、用意したsamplerの機能のうちQUBO形式の組合せ最適化問題を解く関数です。この引数として、先ほど用意したQUBO行列を入れます。
num_readsは非常にユニークなオプションで、何度結果を読み出すかを指定するところです。
結果を受け取ったsamplesetには、指定した組合せ最適化問題をnum_readsで指定した回数分解いた答えが格納されています。

In [ ]:
print(sampleset.record)

結果の一番目を取り出したいときはsampleset.record[0]として、
$x_i$などの結果を知りたいときは、sampleset.record[0][0]から取り出すことができます。
さらにコスト関数の値を知りたいときはsampleset.record[0][1]
その答えが何度出てきたのか頻度を知りたいときはsampleset.record[0][2]で調べることができます。

In [ ]:
sampleset.record[0][0]

簡単に利用できることがご理解いただけましたかね。で
は次に行ってみよう！

## **実感の湧く問題を解いてみよう**

## 例１：分割問題

上記の例はでたらめなQUBO行列によるものであったので、あまり解いたとしても実感が湧かない。

そこで少し具体的に意味のある問題を解いてみましょう。


**いくつかの荷物があり、それぞれには重さが異なるものとします。**

**それを運ぶ2人がいて、重さが均等になるようにその荷物を2つのグループに分けたい。**

どのようにしたら良いでしょうか？

ここで考えなければならないのは**QUBO行列を作る**ことです。
ただしいきなり$Q_{ij}$のことを思っても何も思い浮かばないものです。
そこで重要となるのが**数式によるモデリング**です。
ここが量子アニーリングの研究開発を行う上での成長ポイントです。
量子そのものの前に、数理モデリングに挑戦する必要があるのです。

重さを持ついくつかの荷物があるというのだから、その重さを$w_i$としましょう。
$N$個あるとして、合計した重さは$W=\sum_{i=1}^N w_i$です。

（こうやって何も与えられていないところで**自分で文字式を立てる**ところから訓練です）


2人のうちAさんがその荷物を取る場合$x_i=1$として、取らない場合は$x_i=0$とすると、
Aさんが持つ荷物の重さの合計は、
\begin{equation}
W_A = \sum_{i=1}^N w_i x_i
\end{equation}
逆にBさんは残りの荷物を持つので
\begin{equation}
W_B = W - W_A
\end{equation}
となります。
これらが等しくなるというのだから、
\begin{equation}
W_A - W_B
\end{equation}
という引き算をしたズレが$0$になれば完璧です。
もしくは非常に小さいものとなってくれれば嬉しい。
ただ$W_A$がわずかに大きくても仕方ないし、小さくても仕方ない。正負の値どちらでも良いからとにかく**ズレの大きさ**ができるだけ小さいことが望ましいというわけです。
できるだけ小さい、すなわちズレの大きさが最小になるような組み合わせを見つければ良いですよね。
そうするとコスト関数として、次のようなものを考えてみましょう。
\begin{equation}
E({\bf x}) = \left( W_A - W_B \right)^2 = \left( 2W_A - W \right)^2
\end{equation}
ここに$W_A$の具体的な形として先ほど準備しておいた形を入れてみましょう。
\begin{equation}
E({\bf x}) = \left( 2\sum_{i=1}^N w_i x_i - W \right)^2
\end{equation}



何か近い形になってきましたね。二乗をするというのは同じものを掛け算するという意味です。
シグマ記号は嫌らしいけれども意味はとにかく足し算をするというものでした。
下にある$i=1$は$i$という文字を$1$から動かして上にある$N$まで変えて足し算してくださいね、ということです。
だったら$i$という文字は仮置きをしているだけですから、別の文字を使っても良いですね。


\begin{equation}
E({\bf x}) = \left( 2\sum_{i=1}^N w_i x_i - W \right)\left( 2\sum_{j=1}^N w_j x_j - W \right)
\end{equation}

この掛け算を展開してみましょう。

\begin{equation}
E({\bf x}) = 4\sum_{i=1}^N\sum_{j=1}^N w_iw_j x_ix_j - 2W\sum_{i=1}^N w_i x_i  - 2W\sum_{j=1}^N w_j x_j + W^2
\end{equation}

ここで第二項と第三項で同じ和が2つ出ていますので、まとめておきましょう。
\begin{equation}
E({\bf x}) = 4\sum_{i=1}^N\sum_{j=1}^N w_iw_j x_ix_j - 4W\sum_{i=1}^N w_i x_i + W^2
\end{equation}


$x_ix_j$という部分が出てきましたね。その係数は$4w_iw_j$で、この部分がQUBO行列に相当します。$x_i$が$0$と$1$のどちらかの値を持つことから、$x_i$が$x_ix_i$と同じことを思いつくと、第二項は、$-4W\sum_{i=1}^N w_i x_ix_i$という意味で、QUBO行列のうち添え字が同じ$i$と$i$のとき、対角成分のことを示していることがわかります。
これらの事実からQUBO行列を作るプログラムの発想ができます。

In [ ]:
N = 10
w = np.random.rand(N)

まず例えば$N=10$個の荷物について、その重さを適当な乱数で決めることにしましょう。

np.random.rand()で$0$から$1$の範囲にある適当な数値が出ます。

この係数からQUBO行列を作ります。
まず最初に全重量の計算です。

In [ ]:
W = ---- HERE ----

numpyのsum()を利用すれば全重量の合計が計算できます。

In [ ]:
Q = ---- HERE ----

まずはQUBO行列を格納する場所を用意しましょう。np.zeros()はその名の通り、全成分を0で埋めたものを並べて作ります。これを.reshape(N,N)でN×Nの行列に整形します。

In [ ]:
---- HERE ----

まず第一項の計算をここで実行しています。for文を使って、iとjを動かしながら、$Q_{ij}$すなわちQ[i][j]に値を入れていきます。その値は$4w_iw_j$ですから、その結果を入れていきます。
次に第二項をQUBO行列の対角成分に追加しましょう。

In [ ]:
---- HERE ----

ここで注意して欲しいのが、第一項で計算した結果に追加するので、元からあるQ[i][i]に引き算をするようにしています。

これでQUBO行列の作成完了です。
すでにD-Waveマシンの利用準備は整っておりますので、QUBO行列を投入するだけです。

In [ ]:
sampleset = sampler.sample_qubo(Q, num_reads=10)

In [ ]:
print(sampleset.record)

結果が出てきましたね。それでは一番結果の良かった一番目の結果を抜き出してみましょう。

In [ ]:
x = sampleset.record[0][0]

結果を示す$x_i$の数列をxに格納しました。
これから$W_A$と$W_B$を計算してみましょう。


In [ ]:
print(np.dot(w,x))

In [ ]:
print(W-np.dot(w,x))

ほぼ揃った答えがうまく出せたでしょうか？

## **(補足１) 他のソルバーを利用してみよう！**

D-Wave Systems社の量子アニーリングマシンは、QPUの成長とともに扱える問題の規模を大きくしてきました。
最近ではハイブリッドソルバーの登場により、飛躍的にその規模を大きくしました。
純粋に量子現象を利用した計算だけではないものの、古典的な計算アルゴリズムを巧みに利用して、計算性能も向上させているため大規模な問題を解く上では有用です。
その利用方法についても容易であり、以下のようなコードでsamplerを書き換えるだけです。



In [ ]:
from dwave.system import LeapHybridSampler
sampler = LeapHybridSampler(solver='hybrid_binary_quadratic_model_version2', token=token, endpoint=endpoint)

ソルバーのオプションとして、solver='hybrid_binary_quadratic_model_version2'を選ぶだけでOKです。注意としてはハイブリッドソルバーを利用する場合には、エンベッドの必要がないことです。ここで用意したsamplerを使って直ちに結果を得ることができます。

In [ ]:
sampleset = sampler.sample_qubo(Q)

注意としてハイブリッドソルバーは、オプションとしてnum_readsを持ちません。
ハイブリッドソルバーは**最大で100万変数**のものを扱うことができる（扱うことのできるQUBO行列には全ての値が詰まっていないなどの条件はあります）


In [ ]:
print(sampleset.record)

場合によっては上記の純粋な量子現象のみのQPUによる結果よりも良好な結果を得たのではないでしょうか。これがD-Wave Systems社の用意したハイブリッドソルバーの威力です。
他にもD-Wave Systems社が用意したソルバーとして、**整数値を利用することのできるhybrid_discrete_quadratic_model_version1**があります。

## **(補足２) QUBO行列の設定方法について**

QUBO行列はだんだんと巨大なものになってくると、そのデータ転送にも時間がかかるようになり、せっかくの量子アニーリングマシンのパワーを台無しにしてしまうことがあります。
データの転送量を抑えるためにも、不要な成分についてはその情報を送らないようにしておくと良いでしょう。
python上ではnumpyによるarray形式で行列を扱うことが多いのですが、代わりにdict形式でデータを送りましょう。


In [ ]:
Qdict = {}
for i in range(N):
  for j in range(N):
    if Q[i][j] != 0.0:
      Qdict[(i,j)] = Q[i][j]

dict形式の初期化は{}で中身は空欄にしておくだけです。
Qdict[(i,j)]のように(i,j)でどの成分に値があるのかを指定して、その値を代入します。
ここではif文を使い、ゼロではないところだけ、Qdictのデータとして格納しています。
出来上がったものを確認したい場合にはQdictとそのまま打って実行したり、print(Qdict)と実行してみましょう。

In [ ]:
print(Qdict)

どこの成分に重要な非零の行列成分があるのかを指定する形になっています。
上記の問題では非零成分のない問題になっていますので影響はさほどありませんが、
基本的なテクニックとして知っておくと良いでしょう。

（正直この入力の違いだけでハイブリッドソルバーなどでは如実に性能が変わります）

## **(補足３) シミュレータを活用しよう**

さて上記のように用意した量子アニーリングマシンは、利用回数には制限があり、大事に使いたいところでしょう。演習の際には豊富なマシンタイムを利用することのできるAPI tokenを発行する予定ですが、講義の間の試し利用の場合には、代わりになるシミュレータを利用すると良いでしょう。
その一つが**株式会社Jijの開発するOpenJij**です。

OpenJijは量子アニーリングマシンのシミュレータを搭載するオープンソースソフトウェアです。


基本的な利用方法は、これまでと同じようにQUBO行列を作ったのちにsamplerに投入するだけです。その際にOpenJijのsamplerを利用します。

その前にまずはOpenJijのインストールが必要です。

In [ ]:
pip install openjij

再びpip installを利用して、OpenJijをインストールします。
その後にimport SQASamplerを実行してsamplerの準備を行いましょう。

In [ ]:
from openjij import SQASampler
sampler = SQASampler()

準備はこれだけです。SQAというのはシミュレーテッド量子アニーリングというもので、
**量子モンテカルロ法**という計算技術を活用して、**量子アニーリングのシミュレーション**を行っています。
オプションでそのシミュレーションのパラメータ等を設定することができますが、とりあえず前に進めていきましょう。

In [ ]:
sampleset = sampler.sample_qubo(Qdict, num_reads=10)

In [ ]:
print(sampleset.record)

OpenJijではdict形式で問題を受け付けますので、注意してください。

（先程の手順でnumpy array形式だったものをdict形式に直しておきましょう）

結構いい答えが出てきたのではないでしょうか。これでとりあえずは十分ですよね。
ただ問題のサイズが大きくなるにつれて結果が次第に悪化してくることがありますのでご注意を。
それはパラメータの設定で改善する可能性があります。